# **[:+:] -- Feature Selection + Nearest Neighbor -- [:+:]**

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
#import seaborn as seab
from itertools import chain, combinations
#from scipy.spatial import minkowski_distance
from numba import jit, cuda
from timeit import default_timer as timer
from math import comb
# from itertools import combinations

# [:::] --- TodDo  ---
# - [x] Find "default Rate" : size(most common class)/size(dataset)
# - [x] Create Minkowski Distance function for two points
# - [x] Find nn of a single data point using only one feature
# - [x] Get rid of spurrious percision
# - [x] For small data, find nn of every data point using one feature + max accuracy
# - [x] For small data, find nn of every data point using all features + max accuracy
# - [x] Complete Forward selection for small datas set
# - [x] GPU Multithreading for Classifier
# - [x] Leave-One-Out Cross Validation
# - [x] Complete Backward Elimination for small data set
# - [x] Forward selection methods for large data set
# - [x] Forward Selection validation for both datasets
# - [x] Data Processing funtion
# - [] Backelimination for large Data
# - [] Data Viz
# - [] Report
# - [] User UI
# - [] Third Feature
# - [] Apend Data Processing function with distance matrix formation
# - [] multithread everything else
# - [] Find more brown m&ms
# - [] Clean up the code

### **[+]** Read and Process Data

In [46]:
#Read data from the text files and set as a pandas data frame
lilData = pd.read_csv('CS170_Small_Data__19.txt', sep="  ", engine='python', header=None)
bigData = pd.read_csv('CS170_Large_Data__3.txt', sep="  ", engine='python', header=None)

print("\nConvert to numpy array to enable multithreading later:")
lilData= lilData.to_numpy()
bigData= bigData.to_numpy()
print("column size after conversion: " + str(len(data[0])))
lilData= lilData.transpose()
bigData= bigData.transpose()
print("column size after transposition: " + str(len(data[0])))
print("Dimensions after transposition: " + str(data.shape))
print(type(lilData[0][0]))


data= lilData
#print all the data:
#print(lilData)


#----print just the 'label' column (and indeces):
print("The first 4 entries of the first column: data[colIdx][rowIdx]:")
print(data[0][0:5])
print(type(data[0][0]))


print("\nOriginal Matrix Dimensions:")
lil_rowXcol= data.shape
print(lil_rowXcol)


print("Data Head:")
print(data.head())

print("MetaData: ")
lilData.describe()



print('Occurrence counts of classes:')
# count occurrences of the class column
occur = lilData.groupby([0]).size()
# display occurrences of a particular column
display(occur)





The first 4 entries of the first column: data[colIdx][rowIdx]:
0    2.0
1    1.0
2    2.0
3    2.0
4    2.0
Name: 0, dtype: float64
<class 'numpy.float64'>

Original Matrix Dimensions:
(500, 7)
Data Head:
     0         1         2         3         4         5         6
0  2.0 -1.101866 -0.782026  0.552502  0.454685  1.132363  1.135458
1  1.0  0.928979 -0.169694  1.465293 -1.591929  0.144808  0.162709
2  2.0  1.123118 -1.384730 -0.903598  0.692522  0.669263 -0.142156
3  2.0  0.816617 -0.043628  1.026966  0.231013 -0.006551  2.316509
4  2.0 -1.159129 -1.341375  0.459997  0.631261 -1.479455  0.520158
MetaData: 
Occurrence counts of classes:


0
1.0     95
2.0    405
dtype: int64


Convert to numpy array to enable multithreading later:
column size after conversion: 7
column size after transposition: 500
Dimensions after transposition: (7, 500)
<class 'numpy.float64'>


### **[+]** Minkowski Distance Calculation

In [47]:
#My (from-scratch)Function takes in indices of two data points and exponent p, returns the minkowski distance between them based on the four dimensions and the exponent
@jit(target_backend='cuda')
def minkowski(data,rowIdx1,rowIdx2,ftSet,p):
    # featSet1= []
    # featSet2= []
    sigma= 0
    for ft in ftSet:
        # featSet1.append(data[ft][rowIdx1])
        # featSet2.append(data[ft][rowIdx2])
        sigma+= ((((data[ft][rowIdx1]-data[ft][rowIdx2])**2))**(1/2))  **(p)
        distance= sigma**(1/p)
    return distance
    # return distance, featSet1, featSet2

# d= minkowski(lilData, rowIdx1=1, rowIdx2= 2, ftSet=(1,2,3,4,5,6), p=2)
# print(d)
# minkowski_distance(d[1],d[2],p=2)

### **[:+:]** Forward Selection **+** Backward Elimination

In [51]:
# @jit(nopython=False)
def generateFtCombos(data, limit=6):
    # features= np.empty(0)
    # ftCombos= np.empty(0)
    features= np.empty(0).astype(int)
    ftCombos= []
    ftRow= data.shape[0]
    for i in range(1,ftRow):
        features= np.append(features, i)
    for ft in range(1, limit):
        for combo in itertools.combinations(features, ft):
            ftCombos.append(combo)
    return features, (ftCombos)






    

# @jit(target_backend='cuda')
def generateDefaultRate(data):
    classCnt= [0,0]
    for pt in data[0]:
        if pt==1:
            classCnt[0]+= 1
        else:
            classCnt[1]+= 1
    defaultRate= max(classCnt)/(classCnt[0]+classCnt[1])
    return defaultRate


# takes a single combo of features, classifies each dp by its nearest neighbor, returns a list of classifications for every dp
@jit(target_backend='cuda')
def classifier(data, ftSet):
    rows= data.shape[1]
    nearestNeighbors= np.empty(0)
    for dp in range(0,rows):
        nnIdx= 0
        nearest= 10000
        classification= 0
        for  neighborIdx in range(0,rows):
            if dp != neighborIdx:
                dist= minkowski(data, dp, neighborIdx, ftSet, p=2)
                if dist < nearest:
                    nnIdx= neighborIdx
                    nearest= dist
        classification= data[0][nnIdx]
        nearestNeighbors= np.append(nearestNeighbors, classification)
    return nearestNeighbors


#  Returns the accuracy of the classifier tested with a combination of features (leave-one-out, k=n)
# @jit(target_backend='cuda')
def crossValidate(data, ftSet):
    classes= classifier(data, ftSet)
    classesL= len(classes)
    correct= 0
    for j in range(0,classesL):
        if classes[j] == data[0][j]:
            correct+=1
    accuracy = correct/classesL
    return accuracy


# @jit(target_backend='cuda')
def forwardSelection(data, prevFtSet=np.asarray([]),finalAccuracy=0, finalSet=np.asarray([])):
    ftSetLen= len(prevFtSet)
    ftRowLength= data.shape[0]
    if ftSetLen>= ftRowLength-1:
        bestFtSet= finalSet
        print("[:+:] ---  Forward Selection Results: The best set of features is " +str(bestFtSet)+ " | Accuracy: %" +str(round(finalAccuracy*100,3))+ "  --- [:+:]")
    else:
        print("[::] -- Forward Selection Search Tree Level "+str(ftSetLen+1)+ " -- [::]" )
        if ftSetLen==0:
            print("        - Testing without features  | Default Rate: %" + str(round(generateDefaultRate(data)*100,3)))
        bestAcc= 0
        bestFtSubset= np.empty(0)
        for ft in range(1, ftRowLength):
            ftSet= np.empty(0)
            if ft not in prevFtSet:
                ftSet= np.append(prevFtSet, ft).astype(int)
                ftAccuracy= crossValidate(data,ftSet)
                print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
                if ftAccuracy>bestAcc: 
                    bestAcc= ftAccuracy
                    bestFtSubset= ftSet
        if bestAcc>finalAccuracy:
            finalSet= bestFtSubset
            finalAccuracy= bestAcc
        print("     [+] -- On level "+str(ftSetLen+1)+ ", we add feature subset " + str(bestFtSubset)+" with an accuracy of %" + str(round(bestAcc*100,3))+ "\n")
        bestFtSet=  forwardSelection(data, bestFtSubset,finalAccuracy, finalSet)
    return bestFtSet



def backwardElimination(data, prevFtSet=np.empty(0), eliminated=np.empty(0), globalAcc=0, globalFts=np.empty(0)):
    ftRowL= data.shape[0]
    if ftRowL<10:
        ftCombos= generateFtCombos(data)[1]
    else:
        # ftSetNew= (generateFeatures(ftRowL))
        # ftCombos= getTriplets(np.asarray(ftSetNew))
        ftCombos= generateFtCombos(data,limit=3)[1]
    
    bestAcc= 0
    bestFts= np.empty(0)
    prevComboL= len(prevFtSet)
    leveL= ftRowL - prevComboL
    elimCandidate= []
    print("[::] -- Backward Elimination Search Tree Level "+str(leveL)+ " -- [::]" )
    if(prevComboL==1):
        defaultRate= generateDefaultRate(data)
        print("After eliminating all features, we  und up with a default rate of: " +str(defaultRate))
        print("\n[:+:] ---  Backward Selection Results: The best set of features is " +str(globalFts)+ " | Accuracy: %" +str(round(globalAcc*100,3))+ "  --- [:+:]")
        return globalFts, globalAcc
    else:
        if(prevComboL==ftRowL-1):
            print("        - Testing with features "+ str(prevFtSet) + " | Accuracy: %" + str(round(crossValidate(data, prevFtSet )*100,3)))
        for i in range(0, len(ftCombos)):
            ftSet= np.empty(0)
            combo= set(ftCombos[i])
            if (len(combo)==(prevComboL-1)):
                commonElmtCnt= len(combo.intersection(eliminated))
                if(commonElmtCnt==0):
                    ftSet= np.append(ftSet, list(combo)).astype(int)
                    ftAccuracy= crossValidate(data, ftSet )
                    print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
                    if ftAccuracy>bestAcc:
                        bestAcc= ftAccuracy
                        bestFts= ftSet
                        elimCandidate= list(set(prevFtSet).difference((combo)))[0]
        eliminated= np.append(eliminated, elimCandidate).astype(int)
        print("     [-] -- On level "+str(leveL)+ ", we eliminate feature ["+ str(elimCandidate)+"], and end up with feature set: " + str(bestFts)+" with an accuracy of %" + str(round(bestAcc*100,3)))
        print("                                        [:-:] Eliminated so far:--->"+ str(eliminated)+ "\n")
        if bestAcc>globalAcc:
            globalAcc= bestAcc
            globalFts= bestFts
        return backwardElimination(data, bestFts,eliminated, globalAcc, globalFts)



def exhaustiveSelection(data):
    bestAcc= 0
    bestFt= np.empty(0)
    for combo in generateFtCombos(data)[1]:
        ftSet= np.empty(0)      
        ftSet= np.append(ftSet, combo).astype(int)
        ftAccuracy= crossValidate(data, ftSet )
        print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAccuracy*100,3)))
        if ftAccuracy>bestAcc:
            bestAcc= ftAccuracy
            bestFt= ftSet
    return bestFt, bestAcc

# @jit(nopython=True)
@jit(target_backend='cuda')
def generateFeatures(ftRowL=0):
    # if ftRowL==0:
    #     ftRowL= data.shape[0]
    features= np.empty(0)
    for i in range(1,ftRowL):
        features= np.append(features, i)
    return features

@jit(target_backend='cuda')
def tripleSplits(triplet):
    trippled= len(triplet)
    size= trippled//3
    triSplit= np.empty([size,3])
    triSplit= np.empty(0)
    for i in range(0,trippled):

        if i%3==0:
            triSplit[i//3][0]= triplet[i-2]
            triSplit[i//3][1]= triplet[i-1]
            triSplit[i//3][2]= triplet[i]
    return triSplit
            
        




@jit(target_backend='cuda')
def generateTriplets(ftRowL, setL=3, prevSet=np.array([]),rezSet= np.empty(0),tripletSet= np.empty(0) ):
    if len(prevSet)==setL:
        #print("return prevset")
        return prevSet
    fts= generateFeatures(ftRowL)
    for ft in fts:
        if ft not in prevSet:
            newFtSet= np.copy(prevSet)
            newFtSet= np.append(newFtSet,ft)
            brandNewFtSet= generateTriplets((ftRowL-1),setL,prevSet=newFtSet, tripletSet=tripletSet)
            #print(brandNewFtSet)
            tripletSet= np.concatenate(( tripletSet,brandNewFtSet))
    #print("return tripletSet")
    return tripletSet



def getTriplets(ftSet,limit=3):
    triplets= []
    for comb in itertools.combinations(ftSet, limit):
        # comb= (np.asarray(comb))
        # triplets= np.append(triplets,comb)
        triplets.append(comb)
    return (triplets)

def bigDataBacklimination(data):
    length= data.shape[0]

    features= (generateFeatures(length))

    quadruplets= getTriplets(np.asarray(features),4)
    triplets= getTriplets(np.asarray(features),3)
    doubles= getTriplets(np.asarray(features),2)
    singles= getTriplets(np.asarray(features),1)

    ftComboList= np.array([quadruplets, triplets, doubles, singles])

    for i in range(0,4):
        print(ftComboList[i][0:2])
    #ftCombos= generateFtCombos(data,limit=3)[1]

    bestGlobalAcc= 0
    bestGlobalSet= np.empty(0)
    ftGlobals= []
    accGlobals= []
    
    exclusion= np.empty(0)
    for comboList in ftComboList:
        bestLocalAcc= 0
        bestLocalSet= np.empty(0)
        for  combo in comboList:
            common= np.intersect1d(combo, exclusion)
            if len(common)==0:
                ftSet= np.empty(0)      
                ftSet= np.append(ftSet, combo).astype(int)
                ftAcc= crossValidate(data, ftSet )
                print("        - Testing with features "+ str(ftSet) + " | Accuracy: %" + str(round(ftAcc*100,3)))
                if ftAcc>bestLocalAcc:
                    bestLocalAcc= ftAcc
                    bestLocalSet= ftSet
        if bestLocalAcc>bestGlobalAcc:
            setDiff= np.setdiff1d(bestGlobalSet,bestLocalSet)
            exclusion= np.append(exclusion, setDiff)
            bestGlobalAcc= bestLocalAcc
            bestGlobalSet= bestLocalSet
            ftGlobals.append(  (bestLocalAcc,3)  )  
            accGlobals.append(  (bestLocalSet,3)  )
            print("     [-] -- we eliminate feature ["+ str(setDiff)+"], and end up with feature set: " + str(bestLocalSet)+" with an accuracy of %" + str(round(bestLocalAcc*100,3)))
            print("                                        [:-:] Eliminated so far:--->"+ str(exclusion)+ "\n")
    print("     [-] -- we eliminate feature ["+ str(setDiff)+"], and end up with feature set: " + str(bestLocalSet)+" with an accuracy of %" + str(round(bestLocalAcc*100,3)))
    defaultRate= generateDefaultRate(data)
    print("After eliminating all features, we  und up with a default rate of: " +str(defaultRate))
    print("\n[:+:] ---  Backward Selection Results: The best set of features is " +str(bestGlobalSet)+ " | Accuracy: %" +str(round(bestGlobalAcc*100,3))+ "  --- [:+:]")
    return ftGlobals, accGlobals






# fuck= generateFtCombos(data)
# print(fuck)
# print(len(fuck))

bigDataBacklimination(lilData)

# print("split triple set:")
# print(len(tripletSet))
# print(tripletSet[0:5])
# featureCombos= generateFtCombos(data)
# ftSet= featureCombos[0]
# ftCombos= featureCombos[1]
# print(ftSet)

# triplets= (generateTriplets(data.shape[0]))
# triplets= np.split(triplets, (len(triplets)/3))
# print(triplets)
# print(len(triplets))





#--------------------------------------------

# start = timer()
# print(forwardSelection(data))
# print("Time without GPU:", timer()-start)

# start = timer()
# print(backwardElimination(data, np.asarray([1,2,3,4,5,6])))
# print("Time without GPU:", timer()-start)


# start = timer()
# print(exhaustiveSelection(data))
# print("Time without GPU:", timer()-start)

C:\Users\richa\AppData\Local\Temp\ipykernel_33392\1522238391.py:222: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ftComboList= np.array([quadruplets, triplets, doubles, singles])


[(1.0, 2.0, 3.0, 4.0), (1.0, 2.0, 3.0, 5.0)]
[(1.0, 2.0, 3.0), (1.0, 2.0, 4.0)]
[(1.0, 2.0), (1.0, 3.0)]
[(1.0,), (2.0,)]
        - Testing with features [1 2 3 4] | Accuracy: %71.8
        - Testing with features [1 2 3 5] | Accuracy: %86.8
        - Testing with features [1 2 3 6] | Accuracy: %71.4
        - Testing with features [1 2 4 5] | Accuracy: %87.0
        - Testing with features [1 2 4 6] | Accuracy: %71.8
        - Testing with features [1 2 5 6] | Accuracy: %91.8
        - Testing with features [1 3 4 5] | Accuracy: %86.4
        - Testing with features [1 3 4 6] | Accuracy: %77.2
        - Testing with features [1 3 5 6] | Accuracy: %89.2
        - Testing with features [1 4 5 6] | Accuracy: %88.2
        - Testing with features [2 3 4 5] | Accuracy: %78.0
        - Testing with features [2 3 4 6] | Accuracy: %70.4
        - Testing with features [2 3 5 6] | Accuracy: %80.8
        - Testing with features [2 4 5 6] | Accuracy: %78.8
        - Testing with features [3 4 5

([(0.918, 3), (0.944, 3), (0.962, 3)],
 [(array([1, 2, 5, 6]), 3), (array([1, 5, 6]), 3), (array([1, 5]), 3)])

# **[:+:] Notes**
-   The best single feature is #5, with an accuracy of 0.856

# Works Cited:


-   https://www.analyticsvidhya.com/blog/2020/02/4-types-of-distance-metrics-in-machine-learning/

-   https://www.geeksforgeeks.org/pandas-groupby-count-occurrences-in-column/
-   https://www.codingem.com/python-how-to-get-all-combinations-of-a-list/
-   https://www.geeksforgeeks.org/running-python-script-on-gpu/
-   https://www.programiz.com/python-programming/methods/set/intersection
-   https://www.youtube.com/watch?v=nh2IGC_kLWw
